## Using SerpAPI to webscrape google's showtimes scraper

In [1]:
import requests

### Example

In [4]:
url = 'https://serpapi.com/search.json?q=AMC+Barton+Creek+Square+14&location=Austin%2C+Texas%2C+United+States&hl=en&gl=us&api_key=795f9ca654b1031eb177ec72c1b14382c12a781a6db5e4a739c719fb308e47c8'

In [5]:
respons = requests.get(url)

In [18]:
respons.json()['showtimes'][0]['movies'][0]

{'name': 'The Bad Guys',
 'link': 'https://www.google.com/search?gl=us&hl=en&q=&si=AC1wQDAGEQiTPWsvhpPV4-xl97AcUfloRjKp4qat4I8-Kg1HxfTik-_93U1ilcKEvVFgSHkrObhdLco3znZ0GSowTR_4T8j3FRRF1rpro2VFpzEII6hnJrwTJcyqkfSuVQEwz7xALIYzr1dC67ALPyR3FW6QSbVAzcblNTZX4P1KUchXSkMVhkruEYoFf6F2OXiT8RjmoyMI1N5KX2VbLcoVMXt6eDs9viG9IbwkwfI0S_iR9tvMfiisyYdfHan7EtcJd4ZZQsa6Y1x50tOvLM7OWuFMRKd4Gg%3D%3D&sa=X&ved=2ahUKEwjshPfYs8X3AhVjU98KHWcWA84QxyZ6BAgVEBM',
 'showing': [{'time': ['11:30am',
    '12:30pm',
    '2:00pm',
    '3:00pm',
    '4:30pm',
    '5:30pm',
    '7:00pm',
    '8:00pm'],
   'type': 'Standard'}]}

### Applying to Japan

In [20]:
url = 'https://serpapi.com/search.json'

In [21]:
query = 'united cinemas urawa'

In [22]:
params = {
  "q": query, 
  "hl": "en",
  "gl": "us",
  "api_key": "795f9ca654b1031eb177ec72c1b14382c12a781a6db5e4a739c719fb308e47c8"
}

In [23]:
response = requests.get(url, params=params)

In [59]:
showtimes = response.json()['showtimes']

In [106]:
def google_search(query):
    '''
    takes cinema query and returns googles showtimes, a list of dictionaries for each day
    '''
    url = 'https://serpapi.com/search.json'
    params = {"q": query, "hl": "en","gl": "us",
              "api_key": "795f9ca654b1031eb177ec72c1b14382c12a781a6db5e4a739c719fb308e47c8"}
    
    response = requests.get(url, params=params)
    print(f"Response :{response.status_code}")
    showtimes = response.json()['showtimes']
    
    return showtimes

In [32]:
st['showtimes'][0]['day'], st['showtimes'][1]['day'] #showtimes is a list of dictionaries for each day

('Wed', 'Thu')

In [35]:
st['showtimes'][0].keys()

dict_keys(['day', 'date', 'movies'])

In [49]:
st['showtimes'][0]['movies'][0:2] #movies is a list of dictionaries for each movie

[{'name': 'Detective Conan: The Bride Of Halloween',
  'link': 'https://www.google.com/search?ucbcb=1&gl=us&hl=en&q=&si=AC1wQDAGEQiTPWsvhpPV4-xl97AcUfloRjKp4qat4I8-Kg1HxUvLo3gVdcM4Zhd-wA9e_tiTarn73YB7gitgsAIvryAGZHBRbil5g2wROM5AxMyDE6vDSRPBAep2rfWEllDRGgyzGiqiv7E42D0EC3gDacx6sheiV0yh8dIHrGFt6R1FIbc5ekvq_F0RP6jq-bVKYH_c5KVFPgo7V1ZYf58nD5O3YpGVpVey03b67UxqG8AVkWP124bz84avo7LBgTzrMwjG6htvM1C1YmNahkzcC4LMig79PA%3D%3D&sa=X&ved=2ahUKEwi9jcb2s8X3AhWakWoFHZ9OBfEQxyZ6BAgMEA4',
  'showing': [{'time': ['5:55pm', '7:00pm', '8:00pm'], 'type': 'Standard'},
   {'time': ['6:25pm'], 'type': 'IMAX'}]},
 {'name': 'ドクター・ストレンジ／マルチバース・オブ・マッドネス',
  'link': 'https://www.google.com/search?ucbcb=1&gl=us&hl=en&q=&si=AC1wQDDmkSORa3FH6anizpggOJ18HFgTlSHnpaMgvYkKkNrH1d-xkNrAVbnSn1dZ4-7R47TNF_2ttKdB680NQ0KsULENm8dPPRumiFkz41ZvnGBOdamT17uK-99hROxARNXveoabm3IEIyr-r0iqNK63SSflJj3woOq2mCLI2RqcfVexTGEqjMXjeWyTzcwC8oZmTLx4xQXhDdHiioc-Kcsei09Rut1MpngT4dOX5_MauI4VQ5NPXWRp_836p5iX7-Iw9uYCRkD2do4msgpy-PA-WwX6nxVo6xSRCqXuh9lpO

In [53]:
st['showtimes'][0]['movies'][0]['name']

'Detective Conan: The Bride Of Halloween'

In [55]:
st['showtimes'][0]['movies'][0]['showing'] #showing is a list of dictionarys for each type of showing

[{'time': ['5:55pm', '7:00pm', '8:00pm'], 'type': 'Standard'},
 {'time': ['6:25pm'], 'type': 'IMAX'}]

### Extracting relevant info

In [60]:
showtimes


[{'day': 'Wed',
  'date': 'May 4',
  'movies': [{'name': 'Detective Conan: The Bride Of Halloween',
    'link': 'https://www.google.com/search?ucbcb=1&gl=us&hl=en&q=&si=AC1wQDAGEQiTPWsvhpPV4-xl97AcUfloRjKp4qat4I8-Kg1HxUvLo3gVdcM4Zhd-wA9e_tiTarn73YB7gitgsAIvryAGZHBRbil5g2wROM5AxMyDE6vDSRPBAep2rfWEllDRGgyzGiqiv7E42D0EC3gDacx6sheiV0yh8dIHrGFt6R1FIbc5ekvq_F0RP6jq-bVKYH_c5KVFPgo7V1ZYf58nD5O3YpGVpVey03b67UxqG8AVkWP124bz84avo7LBgTzrMwjG6htvM1C1YmNahkzcC4LMig79PA%3D%3D&sa=X&ved=2ahUKEwi9jcb2s8X3AhWakWoFHZ9OBfEQxyZ6BAgMEA4',
    'showing': [{'time': ['5:55pm', '7:00pm', '8:00pm'], 'type': 'Standard'},
     {'time': ['6:25pm'], 'type': 'IMAX'}]},
   {'name': 'ドクター・ストレンジ／マルチバース・オブ・マッドネス',
    'link': 'https://www.google.com/search?ucbcb=1&gl=us&hl=en&q=&si=AC1wQDDmkSORa3FH6anizpggOJ18HFgTlSHnpaMgvYkKkNrH1d-xkNrAVbnSn1dZ4-7R47TNF_2ttKdB680NQ0KsULENm8dPPRumiFkz41ZvnGBOdamT17uK-99hROxARNXveoabm3IEIyr-r0iqNK63SSflJj3woOq2mCLI2RqcfVexTGEqjMXjeWyTzcwC8oZmTLx4xQXhDdHiioc-Kcsei09Rut1MpngT4dOX5_MauI4VQ5NP

#### Dates collected

In [65]:
startday = showtimes[0]['day'] + ' ' + showtimes[0]['date']
endday   = showtimes[-1]['day'] + ' ' + showtimes[-1]['date']

startday, endday

('Wed May 4', 'Thu May 5')

##### function

In [108]:
def start_end(showtimes):
    '''
    takes showtimes dictionary given by googlesearch and returns tuple of startday and endday
    '''
    startday = showtimes[0]['day'] + ' ' + showtimes[0]['date']
    endday   = showtimes[-1]['day'] + ' ' + showtimes[-1]['date']
    
    return (startday,endday)

In [109]:
start_end(st)

('Fri May 6', 'Thu May 12')

#### List of movies showing each day

In [77]:
showtimes[0]

{'day': 'Wed',
 'date': 'May 4',
 'movies': [{'name': 'Detective Conan: The Bride Of Halloween',
   'link': 'https://www.google.com/search?ucbcb=1&gl=us&hl=en&q=&si=AC1wQDAGEQiTPWsvhpPV4-xl97AcUfloRjKp4qat4I8-Kg1HxUvLo3gVdcM4Zhd-wA9e_tiTarn73YB7gitgsAIvryAGZHBRbil5g2wROM5AxMyDE6vDSRPBAep2rfWEllDRGgyzGiqiv7E42D0EC3gDacx6sheiV0yh8dIHrGFt6R1FIbc5ekvq_F0RP6jq-bVKYH_c5KVFPgo7V1ZYf58nD5O3YpGVpVey03b67UxqG8AVkWP124bz84avo7LBgTzrMwjG6htvM1C1YmNahkzcC4LMig79PA%3D%3D&sa=X&ved=2ahUKEwi9jcb2s8X3AhWakWoFHZ9OBfEQxyZ6BAgMEA4',
   'showing': [{'time': ['5:55pm', '7:00pm', '8:00pm'], 'type': 'Standard'},
    {'time': ['6:25pm'], 'type': 'IMAX'}]},
  {'name': 'ドクター・ストレンジ／マルチバース・オブ・マッドネス',
   'link': 'https://www.google.com/search?ucbcb=1&gl=us&hl=en&q=&si=AC1wQDDmkSORa3FH6anizpggOJ18HFgTlSHnpaMgvYkKkNrH1d-xkNrAVbnSn1dZ4-7R47TNF_2ttKdB680NQ0KsULENm8dPPRumiFkz41ZvnGBOdamT17uK-99hROxARNXveoabm3IEIyr-r0iqNK63SSflJj3woOq2mCLI2RqcfVexTGEqjMXjeWyTzcwC8oZmTLx4xQXhDdHiioc-Kcsei09Rut1MpngT4dOX5_MauI4VQ5NPXWRp_836

In [87]:
daymovies = {}
for day in showtimes:
    print()
    daymovies[day['day'].lower] = [movie['name'] for movie in day['movies']]


In [99]:
daymovies = {day['date'].split()[1] : [movie['name'] for movie in day['movies']] for day in showtimes }
#makes a dictionary with each date value as key and a list of movies that day as the value

In [105]:
daymovies

{'4': ['Detective Conan: The Bride Of Halloween',
  'ドクター・ストレンジ／マルチバース・オブ・マッドネス',
  '劇場版ラジエーションハウス',
  'ホリック xxxHOLiC',
  'Free! – the Final Stroke – the first volume',
  "C'mon C'mon"],
 '5': ['Detective Conan: The Bride Of Halloween',
  'Fantastic Beasts: The Secrets of Dumbledore',
  'ドクター・ストレンジ／マルチバース・オブ・マッドネス',
  '劇場版ラジエーションハウス',
  'Crayon Shin-chan: Mononoke Ninja Chinpūden',
  'ホリック xxxHOLiC',
  'Free! – the Final Stroke – the first volume',
  'Sing 2',
  "Doraemon: Nobita's Little Star Wars 2021",
  'Jujutsu Kaisen 0',
  '映画 おそ松さん',
  'Tonbi',
  '余命10年',
  "C'mon C'mon"]}

##### function

In [110]:
def movies_by_day(showtimes):
    '''
    takes showtimes dict and returns a dictionary- keys are each day, values are lists of movie titles
    '''
    moviesbyday = {day['date'].split()[1] : [movie['name'] for movie in day['movies']] for day in showtimes }
    return moviesbyday

In [111]:
movies_by_day(st)

{'6': ['Detective Conan: The Bride Of Halloween',
  'Doctor Strange in the Multiverse of Madness',
  'Fantastic Beasts: The Secrets of Dumbledore',
  '劇場版ラジエーションハウス',
  'Crayon Shin-chan: Mononoke Ninja Chinpūden',
  'ホリック xxxHOLiC',
  'Free! – the Final Stroke – the first volume',
  'Sing 2',
  'Jujutsu Kaisen 0',
  'チェリまほ THE MOVIE ～３０歳まで童貞だと魔法使いになれるらしい～'],
 '7': ['Detective Conan: The Bride Of Halloween',
  'Doctor Strange in the Multiverse of Madness',
  'Fantastic Beasts: The Secrets of Dumbledore',
  '劇場版ラジエーションハウス',
  'Crayon Shin-chan: Mononoke Ninja Chinpūden',
  'ホリック xxxHOLiC',
  'Free! – the Final Stroke – the first volume',
  'Sing 2',
  'Jujutsu Kaisen 0',
  "Doraemon: Nobita's Little Star Wars 2021",
  '映画 おそ松さん',
  'チェリまほ THE MOVIE ～３０歳まで童貞だと魔法使いになれるらしい～',
  'Two for the Road'],
 '8': ['Detective Conan: The Bride Of Halloween',
  'Doctor Strange in the Multiverse of Madness',
  'Fantastic Beasts: The Secrets of Dumbledore',
  '劇場版ラジエーションハウス',
  'Crayon Shin-chan: Monono

#### list of all movies showing

In [126]:
movie_set = {movie['name'] for movie in day['movies'] for day in showtimes}

In [128]:
list(movie_set)

['Tonbi',
 "Doraemon: Nobita's Little Star Wars 2021",
 'ホリック xxxHOLiC',
 'Detective Conan: The Bride Of Halloween',
 'Fantastic Beasts: The Secrets of Dumbledore',
 'Free! – the Final Stroke – the first volume',
 'Jujutsu Kaisen 0',
 '余命10年',
 "C'mon C'mon",
 'Sing 2',
 'ドクター・ストレンジ／マルチバース・オブ・マッドネス',
 '映画 おそ松さん',
 '劇場版ラジエーションハウス',
 'Crayon Shin-chan: Mononoke Ninja Chinpūden']

In [129]:
def movie_list(showtimes):
    '''
    goes through showtimes and returns a list of each unique movie
    '''
    movie_set = {movie['name'] for movie in day['movies'] for day in showtimes}
    return list(movie_set)